In [11]:
# import sys
# sys.path.append('../causal_conv1d/')
# sys.path.append('../mamba/')

In [12]:
from video_mm.models.backbones.videomamba import build_videomamba

In [22]:
model_config = dict(
    model_cls="UMT_VIDEOMAMBA",
    vision_encoder=dict(
        # backbone
        name="videomamba_middle",
        img_size=224, 
        patch_size=16, 
        depth=32, 
        embed_dim=576, 
        drop_path_rate=0.25,
        ssm_cfg=None, 
        norm_epsilon=1e-5, 
        fused_add_norm=True,
        rms_norm=True, 
        residual_in_fp32=True,
        bimamba_type="v2",
        pool_type="cls+avg",
        kernel_size=1,
        num_frames=8,
        ckpt_num_frame=8,
        use_checkpoint=False,
        checkpoint_num=0,
        clip_decoder_embed_dim=576,
        clip_output_dim=512,
        clip_norm_type='l2',
        clip_return_layer=1,
        clip_student_return_interval=1,
        pretrained=None, #"../../biases/pretrained/videomamba/videomamba_m16_25M_f8_res224.pth",
        # clip teacher
        clip_teacher="none",
        clip_img_size=224,
        clip_return_interval=1,
        # mask
        video_mask_type="none",
        video_mask_ratio=0.,
        video_double_mask_ratio=0.,
        image_mask_type="none",
        image_mask_ratio=0.,
        image_double_mask_ratio=0.,
        # for ret
        keep_temporal=True,
    ),
    text_encoder="${TextEncoders[${text_enc}]}",
    multimodal=dict(enable=True),
    embed_dim=512,
    temp=0.07,
)

from argparse import Namespace
# model_config = Namespace(**model_config)

import json

# https://stackoverflow.com/questions/66208077/how-to-convert-a-nested-python-dictionary-into-a-simple-namespace
class _DotDict(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


def dot(data=None):
    if data is []:
        return []
    return json.loads(json.dumps(data), object_hook=_DotDict) if data else _DotDict()

model_config = dot(model_config)

In [41]:
model = build_videomamba(model_config, add_pool_norm=False)

    # if "bert" in config.model.text_encoder.name:
    #     tokenizer = BertTokenizer.from_pretrained(config.model.text_encoder.pretrained, local_files_only=True)
    # else:
    #     raise ValueError(f"Not supported text encoder.")

    # model = model_cls(config=config, tokenizer=tokenizer, is_pretrain=pretrain)

In [45]:
model.load_state_dict(new_checkpoint, strict=False)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['pool_norm.weight', 'pool_norm.bias'])

In [28]:
model.cuda()

x = torch.zeros(8, 3, 8, 224, 224).cuda()
y = model(x)

In [48]:
torch.isnan(y[0]).sum() #y[0].shape

tensor(0, device='cuda:0')

In [47]:
576 / (768 / 12)

9.0

In [43]:
import torch 

checkpoint = torch.load('../../biases/pretrained/videomamba/videomamba_m16_25M_f8_res224.pth')
# checkpoint = torch.load('../../biases/pretrained/videomamba/videomamba_m16_k400_mask_pt_f8_res224.pth')

# checkpoint.keys()

In [44]:
new_checkpoint = {}
for k, v in checkpoint.items():
    if 'vision_encoder.' in k:
        new_checkpoint[k[len('vision_encoder.'):]] = v

# new_checkpoint.keys()

In [8]:
model_config

{'model_cls': 'UMT_VIDEOMAMBA',
 'vision_encoder': {'name': 'videomamba_middle',
  'img_size': 224,
  'patch_size': 16,
  'depth': 32,
  'embed_dim': 576,
  'drop_path_rate': 0.25,
  'ssm_cfg': None,
  'norm_epsilon': 1e-05,
  'fused_add_norm': True,
  'rms_norm': True,
  'residual_in_fp32': True,
  'bimamba_type': 'v2',
  'pool_type': 'cls+avg',
  'kernel_size': 1,
  'num_frames': '${num_frames}',
  'ckpt_num_frame': 8,
  'use_checkpoint': False,
  'checkpoint_num': 0,
  'clip_decoder_embed_dim': 576,
  'clip_output_dim': 512,
  'clip_norm_type': 'l2',
  'clip_return_layer': 1,
  'clip_student_return_interval': 1,
  'pretrained': 'your_model_path/videomamba_m16_k400_mask_pt_f8_res224.pth',
  'clip_teacher': 'none',
  'clip_img_size': 224,
  'clip_return_interval': 1,
  'video_mask_type': 'none',
  'video_mask_ratio': 0.0,
  'video_double_mask_ratio': 0.0,
  'image_mask_type': 'none',
  'image_mask_ratio': 0.0,
  'image_double_mask_ratio': 0.0,
  'keep_temporal': True},
 'text_encoder'